In [4]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *

import warnings
warnings.filterwarnings('ignore')

In [5]:
key = 'Infected'
var_list = ['up1', 'up2', 'down1', 'down2']
cities = ['강원', '경기', '경남', '경북', '광주', '대구', 
          '대전', '부산', '서울', '세종', '울산', '인천',
          '전남', '전북', '제주', '충남', '충북']
models = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
          'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 
          'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

In [8]:
var_num = 0
dic_total_model_final = {}
dic_model_rank = {}
dic_city_model_rank = {}

for name in models:
    dic_model_rank[name] = 0
    
dic_model_no_rank = dic_model_rank.copy()

for var in var_list:
    
    path = f'Data/Stricgency/{key}/{var}/*.csv'
    dic_files = Load_files.load_files(path, -17, -15)

    dic_hyperparameter = {}
    for city in cities:
        path = f'hyperparameter/Stricgency/{key}/{var}/{city}/*.pkl'
        files = glob.glob(path)
        hyperparameter = {}
        for file in files:
            if var_num < 2:
                hyperparameter[file[49:-4]] = load_hyperparameter(file)
            else:
                hyperparameter[file[53:-4]] = load_hyperparameter(file)
                
        dic_hyperparameter[city] = hyperparameter
        
    city_predicted = {}

    for city in cities:

        dic_model = load_model_multiple(dic_hyperparameter, 'Stricgency', key, var, city)
        data = dic_files[city]
        df = Prepare_df.processing(data, 'stdDay', 'defCnt')

        x = df.iloc[:, 0:]
        y = df.iloc[:,:1]

        ms = MinMaxScaler()
        ss = StandardScaler()

        ss.fit(x)
        ms.fit(y)

        predicted_values = {}
        num_model = 0
        for name, state in dic_model.items():
            batch_size = 64
            if num_model < 6:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 138, 60, 1, batch_size, 'mto')
                label_y, predicted = predict_mto(state, df, x_ss, y_ms)
                predicted_values[name] = [label_y, predicted]
            else:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 132, 60, 7, batch_size, 'mtm')
                label_y, predicted, first_label_y, first_predicted, all_predicted = predict_mtm(state, df, x_ss, y_ms, 7, 0.5, device)
                predicted_values[name] = [label_y, predicted, first_label_y, first_predicted, all_predicted]

            num_model += 1

        city_predicted[city] = predicted_values

    dic_best_model = {}
    dic_total_model = {}

    for city in cities:

        actual_data = city_predicted[city]['RNN'][0]
        num_model = 0
        dic_model = {}

        for name, file in city_predicted[city].items():
            if num_model < 6:
                predicted_data = file[1]
            else:
                predicted_data = file[4]

            rmse_val = rmse(actual_data[-7:], predicted_data[-7:])
            mape_val = mape(actual_data[-7:], predicted_data[-7:])
            dic_model[name] = [rmse_val, mape_val, actual_data, predicted_data]

            num_model += 1

        dic_total_model[city] = dic_model
    dic_total_model_final[var] = dic_total_model
    var_num += 1
    
    dic_var_model_rank = {}
    for city in cities:
        dic_model_rank_sub = dic_model_no_rank.copy()
        rmse_rank = list(dict(sorted(dic_total_model[city].items(), key = lambda x:x[1][0])).keys())
        for m in rmse_rank:
            dic_model_rank[m] += rmse_rank.index(m)
            dic_model_rank_sub[m] += rmse_rank.index(m)
#         mape_rank = list(dict(sorted(dic_total_model[city].items(), key = lambda x:x[1][1])).keys())
#         for m in mape_rank:
#             dic_model_rank[m] += mape_rank.index(m)
#             dic_model_rank_sub[m] += mape_rank.index(m)
#         dic_var_model_rank[city] = list(dict(list(sorted(dic_model_rank_sub.items(), key = lambda x:x[1]))).keys())
    dic_city_model_rank[var] = dic_var_model_rank

In [10]:
dic_city_model_rank['up1']

{}

In [5]:
dict(sorted(dic_model_rank.items(), key = lambda x:x[1]))

{'seq2seq_RNN': 492,
 'seq2seq_GRU': 520,
 'seq2seq_LSTM': 654,
 'LSTM': 668,
 'GRU': 701,
 'BiLSTM': 715,
 'BiGRU': 722,
 'BiRNN': 777,
 'RNN': 835,
 'seq2seq_BiGRU': 921,
 'seq2seq_BiLSTM': 976,
 'seq2seq_BiRNN': 995}

In [6]:
model_rank_list = list(dict(sorted(dic_model_rank.items(), key = lambda x:x[1])).keys())
model_rank_list

['seq2seq_RNN',
 'seq2seq_GRU',
 'seq2seq_LSTM',
 'LSTM',
 'GRU',
 'BiLSTM',
 'BiGRU',
 'BiRNN',
 'RNN',
 'seq2seq_BiGRU',
 'seq2seq_BiLSTM',
 'seq2seq_BiRNN']

In [7]:
top_model = model_rank_list[:6]
top_model

['seq2seq_RNN', 'seq2seq_GRU', 'seq2seq_LSTM', 'LSTM', 'GRU', 'BiLSTM']

In [8]:
dic_var_diff = {}

for var in var_list:
    path = f'Data/Stricgency/{key}/{var}/*.csv'
    dic_files = Load_files.load_files(path, -17, -15)
    dic_city_diff = {}
    for city in cities:
        data = dic_files[city]
        index_min = np.inf
        dic_model_diff = {}
        print(var, city)
        for name in top_model:
            rmse_val = rmse(dic_total_model_final[var][city][name][2][-7:], dic_total_model_final[var][city][name][3][-7:])
            mape_val = mape(dic_total_model_final[var][city][name][2][-7:], dic_total_model_final[var][city][name][3][-7:])
            diff_val = float(sum(dic_total_model_final[var][city][name][2][-7:] - dic_total_model_final[var][city][name][3][-7:]))
            index_val = dic_city_model_rank[var][city].index(name)
            dic_model_diff[name] = [rmse_val, mape_val, diff_val]
            if index_min > index_val:
                index_min = index_val
                best_rmse = rmse_val
                best_mape = mape_val
                best_diff = diff_val
                best_model = name
                best_predicted = dic_total_model_final[var][city][name][3]
                
            df = pd.DataFrame(np.concatenate([dic_total_model_final[var][city][name][2], 
                                              dic_total_model_final[var][city][name][3]], 
                                             axis = 1), 
                              columns = ['Actual_data', 'Predicted_data'])
            df.index = list(data['stdDay'][-1*len(actual_data):])
            df.to_csv(f"Data/Predicted/Stricgency/{key}/{var}/{city}/{name}.csv", encoding = 'cp949')
        dic_model_diff['best_model'] = [best_model, best_rmse, best_mape, best_diff]
        dic_city_diff[city] = dic_model_diff
        df_best = pd.DataFrame(np.concatenate([dic_total_model_final[var][city][name][2], 
                                               best_predicted], 
                                              axis = 1), 
                              columns = ['Actual_data', 'Predicted_data'])
        df_best.index = list(data['stdDay'][-1*len(actual_data):])
        df_best.to_csv(f"Data/Predicted/Stricgency/{key}/{var}/{city}/best_{best_model}.csv", encoding = 'cp949')
        print(best_model, best_rmse, best_rmse, best_diff)
    dic_var_diff[var] = dic_city_diff
    
with open(f'Data/Predicted/Stricgency/{key}/valid_score.pkl', 'wb') as f:
    pickle.dump(dic_var_diff, f)

up1 강원
GRU 45.06713741741314 45.06713741741314 312.3153076171875
up1 경기
GRU 351.3210559801391 351.3210559801391 2228.193359375
up1 경남
LSTM 58.437548044766345 58.437548044766345 401.843994140625
up1 경북
LSTM 9.569617720316119 9.569617720316119 0.81201171875
up1 광주
GRU 58.50046115269627 58.50046115269627 337.67822265625
up1 대구
GRU 27.58558238407792 27.58558238407792 178.4189453125
up1 대전
seq2seq_RNN 8.880706671699585 8.880706671699585 -4.31781005859375
up1 부산
BiLSTM 68.2367024305377 68.2367024305377 472.886474609375
up1 서울
BiLSTM 180.07044519958015 180.07044519958015 -757.8046875
up1 세종
LSTM 1.7781242249927944 1.7781242249927944 -1.42987060546875
up1 울산
LSTM 64.16902347876855 64.16902347876855 446.37091064453125
up1 인천
LSTM 49.71137152588304 49.71137152588304 321.66015625
up1 전남
GRU 19.354937792288545 19.354937792288545 124.4854736328125
up1 전북
seq2seq_RNN 26.29151641593194 26.29151641593194 171.398681640625
up1 제주
GRU 8.319891604671094 8.319891604671094 -24.5614013671875
up1 충남
LSTM 26.6